In [1]:
# import modules
import numpy as np

# import plotting modules
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

from IPython.display import Latex

%matplotlib inline

from matplotlib import gridspec
from NFW_Z import NFW_Z

## NOTE: The following code uses the LOCAL density within spherical shells.

In [2]:
class GenerateNFWFits:
    
    
    def __init__(self, Mv):
        
        """Initiate the class with a known virial mass of the halo.
        Input: virial mass in solar masses."""
        
        self.Rvir = Mv


    def find_dens_profile(self, raw_radius_data, radius_array, bin_size):
        
        """Derive an AVERAGE dark matter density profile from raw data.
        Inputs:
        1) raw_radius_data: array of the radii at which dark matter particles are found, based on provided data.
        2) radius_array: array of the radii of imaginary spherical shells centered on the galactic center.
        3) bin_size: integer that best matches the image resolution. In kpc.
        Output: density_profile (the AVERAGE dark matter density profile of the data)"""
        
        density_profile = np.zeros(np.size(radius_array))
        i = 0
        for radius_value in radius_array: # local density
            particles = np.where((raw_radius_data > radius_value) & (raw_radius_data < (radius_value + bin_size)))
            how_many_particles = np.size(particles)
            shell_volume = 4/3*np.pi*((radius_value + bin_size)**3 - radius_value**3)
            density_profile[i] = how_many_particles / shell_volume * 500 #500 is 500 M_Sun, the mass per particle
            i += 1
        return density_profile
    
    
    def final_plot(self, Mvir, upper_limit_estimate, increment, redshift, raw_radius_data, radius_array, bin_size):
        
        """Make a ratio plot that compares the actual density profile generated from data with three NFW fits
        based on varying values of M_vir.
        Inputs:
        1) Mvir: either the quoted M_vir or the total particle mass of the entire dwarf. Must be an integer.
        2) upper_limit_estimate: what's the limit at which the increase of the virial mass should stop? Must be an integer.
        3) increment: the increment by which the data's virial mass is increased. Must be an integer.
        4) redshift: the redshift at which the data was taken.
        5) raw_radius_data: array of the radii at which dark matter particles are found, based on provided data.
        6) radius_array: array of the radii of imaginary spherical shells centered on the galactic center.
        7) bin_size, integer that best matches the image resolution. In kpc.
        Output: the ratio plot, of course!"""
        
        # extract the density profile array from find_dens_profile
        density_profile = self.find_dens_profile(raw_radius_data, radius_array, bin_size)
        
        # this does all the plotting
        fig = plt.figure(figsize=(8, 10)) 
        gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) # row, columns, widths

        ax0 = plt.subplot(gs[0]) # top plot
        ax1 = plt.subplot(gs[1]) # bottom plot

        fig.subplots_adjust(hspace=0.1)  

        ax0.loglog(radius_array, density_profile, 'b', lw=3, label="data")

        top_labels = []
        bottom_labels = []

        i = 1

        for virial_mass in range(Mvir, upper_limit_estimate, increment):
            new_halo = NFW_Z(virial_mass)
            NFW_profile = new_halo.rho(redshift, radius_array)
            r = np.random.random()
            g = np.random.random()
            b = np.random.random()
            ax0.loglog(radius_array, NFW_profile, color=(r, g, b), linestyle='-.', lw=2)
            ax1.plot(radius_array, density_profile / NFW_profile, color=(r, g, b), linestyle='-.', lw=2)

            # top legends
            label_plot1 = mpatches.Patch(color=(r, g, b), label='NFW' + str(i) + ': $M_{vir}$ = ' + \
                                         format(virial_mass, '.2e') + " $M_\odot$")
            top_labels.append(label_plot1)

            # bottom legends
            label_plot2 = mpatches.Patch(color=(r, g, b), label='data/NFW' + str(i))
            bottom_labels.append(label_plot2)
            i += 1

        ax0.minorticks_on()

        ax0.set_title('Plot Title', fontsize=20)
        ax0.set_xticklabels([]) # ignore the x label
        ax0.set_ylabel(r' $\rho$ [M$_\odot$/kpc$^3$] ', fontsize=20)
        ax0.tick_params(axis='both', which='both', direction='in',length=6, width=2)

        ax0.legend(handles=top_labels, bbox_to_anchor=(1.02, 1.01), loc='upper left')
        ax1.legend(handles=bottom_labels, bbox_to_anchor=(1.02, 0), loc='lower left')

        plt.xscale('log')

        ax1.set_ylabel('Ratio',fontsize=20)
        ax1.set_xlabel('r (kpc)', fontsize=20)

        ax1.grid()
        ax1.minorticks_on()
        ax1.tick_params(axis='both', which='both', direction='in',length=6, width=1)

        plt.rcParams.update({'font.size': 15})